In [1]:
import sys
sys.path.append('../')
import os

In [2]:
from application.review_obj_class import ReviewObj
from application.namespan_class import NameSpanGenerator
import pandas as pd
%pprint

Pretty printing has been turned OFF


In [3]:
directory = "../../aps_reviews_1000/"
filenames = os.listdir(directory)
reviews = ((file.split('.')[0], open(directory + file).read()) for file in filenames)

In [4]:
review_list = [ReviewObj(file, txt) for (file, txt) in reviews]

In [5]:
ex = NameSpanGenerator.generate(review_list[20])

In [6]:
[x.collocates for x in ex.spans]

[['tribute', 'to', 'Dr._Gladden', "'s", 'zealous'], ['writers', 'of', 'Dr._Gladden', "'s", 'school'], ['probably', 'what', 'Dr._Gladden', 'means', ','], ['country', '.', 'Dr._Gladden', 'is', 'not']]

In [7]:
list(set((x for x in review_list[0].cleaned_text[:50].split() if x.isalnum())))

['Rl', 'New']

In [8]:
[x.name for x in ex.spans]

[Dr. Gladden, Dr. Gladden, Dr. Gladden, Dr. Gladden]

In [9]:
for x in review_list:
    x = NameSpanGenerator.generate(x)

In [10]:
example_reviews = [x for x in review_list if (len(x.person_names) > 3) and (len(x.pub_names) > 1)]

In [11]:
len(example_reviews)

77

In [12]:
[x.name for x in example_reviews[0].spans]

[Dr. Leopold Damrosch, Mr. Andrew Carnegie, Dr. LeopOld Damrosch, Prof. Horatio W. Parker, Mr. Parker, Lord. The, Oratorio Society, Oratorio Society, Mendelssohn Society, Symphony Society]

In [13]:
name_indices = [x.name.review_loc_toks for x in example_reviews[0].spans]

In [14]:
name_indices

[244, 423, 459, 493, 561, 731, 3, 64, 284, 433]

In [15]:
example_reviews[0].coll_toks_ind[name_indices[0]-2:name_indices[0]+3]

['z873', 'that', 'Dr._Leopold_Damrosch', 'united', 'a']

## Looking through the collocations

In [16]:
review_list[20].spans

[<application.namespan_class.NameSpan object at 0x1a35880390>, <application.namespan_class.NameSpan object at 0x1a35880150>, <application.namespan_class.NameSpan object at 0x1a358802d0>, <application.namespan_class.NameSpan object at 0x1a358803d0>]

In [17]:
[span for spans in (rev.spans for rev in review_list[:2]) for span in spans]

[<application.namespan_class.NameSpan object at 0x1a35874310>, <application.namespan_class.NameSpan object at 0x1a358742d0>, <application.namespan_class.NameSpan object at 0x1a35874390>, <application.namespan_class.NameSpan object at 0x1a35874410>, <application.namespan_class.NameSpan object at 0x1a35874550>, <application.namespan_class.NameSpan object at 0x1a35874590>, <application.namespan_class.NameSpan object at 0x1a358743d0>, <application.namespan_class.NameSpan object at 0x1a358744d0>, <application.namespan_class.NameSpan object at 0x1a35874510>, <application.namespan_class.NameSpan object at 0x1a358745d0>, <application.namespan_class.NameSpan object at 0x1a35874450>, <application.namespan_class.NameSpan object at 0x1a35874490>, <application.namespan_class.NameSpan object at 0x1a35874650>, <application.namespan_class.NameSpan object at 0x1a35874690>, <application.namespan_class.NameSpan object at 0x1a358746d0>, <application.namespan_class.NameSpan object at 0x1a35874710>, <applic

In [18]:
spans = [span for spans in (rev.spans for rev in review_list) for span in spans]

In [19]:
len(spans)

5839

In [20]:
person_spans = [span for span in spans if span.label == 'person']

### Problem situations:
    
+ Okay: 0 1 name(2) 3 4
    + 2 !< 2
+ Bad: X 0 name(1) 2 3
    + 1 < 2
    + **2 - 1 = 1**
+ Bad: X X name(0) 2 3
    + 0 < 2
    + **2 - 0 = 2**

**Pattern: span.name.review_loc_toks < 2**

+ Okay: -5 -4 name(-3) -2 -1
    + 0 1 name(2) 3 4
    + 5 toks, 5 - 3 = 2
    + 2 !> 2
+ Bad: -4 -3 name(-2) -1 X
    + 0 1 name(2) 3
    + 4 toks, 4 - 3 = 1
    + 2 > 1
    + **2 - 1 = 1**
+ Bad: -3 -2 name(-1) X X
    + 0 1 name(2)
    + 3 toks, 3 - 3 = 0
    + 2 > 0
    + **2 - 0 = 2**
+ Bad: -4 -3 name(-2) -1 X
    + 300 301 302(name) 303 X
    + 304 toks, 304 - 3 = 301
    + 302 > 301
    + **302 - 301 = 1**
+ Bad: -3 -2 name(-1) X X
    + 302 303(name) X X
    + 304 toks, 304 - 3 = 301
    + 303 > 301
    + **303 - 301 = 2**

**Pattern: span.name.review_loc_toks > (len(toks)-3)**

In [21]:
# len([span.collocates for span in person_spans if len(span.collocates) < 5])

In [22]:
# [span.collocates for span in person_spans if len(span.collocates) < 5]

In [23]:
# problem_ids = [span.review_id for span in person_spans if len(span.collocates) < 5]

In [24]:
# problem_revs = [rev for rev in review_list if rev.review_id in problem_ids]

In [25]:
# len(problem_revs)

In [26]:
# import copy

In [27]:
# for review in problem_revs:
#     for span in review.spans:
#         toks = copy.deepcopy(review.coll_toks_ind)
#         ni = span.name.review_loc_toks
#         if (ni < 2):
#             print("front:")
#             ftoks = list((2-ni)*"^")
#             ftoks.extend((toks))
#             print(ftoks[0:(ni+3+(2-ni))])
#             print()
#         if (ni > (len(toks)-3)):
#             print("back:")
#             btoks = toks
#             btoks.extend(list((ni-(len(toks)-3)) * "$"))
#             print(btoks[(ni-2):])
#             print()

In [28]:
len(person_spans)

5192

In [29]:
coll_df = pd.DataFrame()

In [30]:
coll_df['review_id'] = list(map((lambda x: x.review_id), person_spans))
coll_df['review_group'] = list(map((lambda x: x.group), person_spans))

In [31]:
person_spans[0].collocates

['Place', 'and', 'Mrs._Canfield', "'s", 'The']

In [32]:
# problem fixed
set([len(span.collocates) for span in person_spans])

{5}

In [33]:
coll_df['preceding_collocates'] = list(map((lambda x: x.collocates[:2]), person_spans))
coll_df['following_collocates'] = list(map((lambda x: x.collocates[3:]), person_spans))

coll_df['coll-2'] = list(map((lambda x: x.collocates[0]), person_spans))
coll_df['coll-1'] = list(map((lambda x: x.collocates[1]), person_spans))
coll_df['name_target'] = list(map((lambda x: x.name), person_spans))
coll_df['coll+1'] = list(map((lambda x: x.collocates[3]), person_spans))
coll_df['coll+2'] = list(map((lambda x: x.collocates[4]), person_spans))

In [37]:
coll_df['all_coll'] = coll_df.preceding_collocates + coll_df.following_collocates

In [38]:
coll_df.head()

,review_id,review_group,preceding_collocates,following_collocates,coll-2,coll-1,name_target,coll+1,coll+2,all_coll
0,137197814,21,"[Place, and]","['s, The]",Place,and,Mrs. Canfield,'s,The,"[Place, and, 's, The]"
1,137197814,22,"[it, is]","['s, own]",it,is,Mr. Anderson,'s,own,"[it, is, 's, own]"
2,137197814,1,"[Years, ago]","[,, writing]",Years,ago,Mr. George Moore,",",writing,"[Years, ago, ,, writing]"
3,137197814,26,"[Matisse, .]","[is, a]",Matisse,.,Mr. Swinnerton,is,a,"[Matisse, ., is, a]"
4,137197814,33,"[house, .]","[seems, to]",house,.,Mr. Hergesheimer,seems,to,"[house, ., seems, to]"


In [39]:
coll_df.preceding_collocates.value_counts()[:10]

[., By]         43
[,, and]        39
[,, by]         33
[,, as]         29
[of, the]       25
[,, ``]         24
[edited, by]    21
[by, the]       19
[,, that]       18
[., But]        18
Name: preceding_collocates, dtype: int64

In [40]:
coll_df.following_collocates.value_counts()[:10]

['s, book]    63
[,, of]       50
[,, who]      49
[,, and]      46
[of, the]     39
['s, ``]      39
['s, work]    36
[,, the]      33
[,, in]       32
[,, ``]       30
Name: following_collocates, dtype: int64

In [41]:
coll_df.all_coll.value_counts()[:10]

[``, says, ,, ``]        6
[., By, ,, M.]           5
[,, by, ,, of]           4
[., By, ., New]          3
[,, by, ,, is]           3
[of, ``, ,, ``]          3
[edited, by, ,, D.D.]    2
[,, by, ., (]            2
[Rev, ., ,, who]         2
[,, of, 's, ``]          2
Name: all_coll, dtype: int64